In [36]:
!pip install python-telegram-bot
!pip install requests
!pip install nest_asyncio


In [37]:
import asyncio
import nest_asyncio
from typing import Final
from telegram import (Update, InlineKeyboardButton, InlineKeyboardMarkup)
from telegram.ext import (Application, CallbackQueryHandler, CommandHandler,
                          MessageHandler, filters, ContextTypes)
from telegram.constants import ParseMode
import requests

nest_asyncio.apply()

In [ ]:
# Replace the strings with your bot's info!
TOKEN: Final = '7163613477:AAHmHBD52_pnCwH94Jujux9HnvZQ1G6uh1o' # NEVER SHARE THIS PUBLICLY
BOT_USER: Final = ''

# List of Module Codes for CS Focus Area Primaries
PRIMARIES_LIST: Final = [
    ['CS3230', 'CS3231', 'CS3236', 'CS4231', 'CS4234'],            # Algo
    ['CS2109S', 'CS3263', 'CS3264', 'CS4243', 'CS4246', 'CS4248'], # AI
    ['CS3241', 'CS3242', 'CS3247', 'CS4247', 'CS4350'],            # Games
    ['CS2107', 'CS3235', 'CS4236', 'CS4230', 'CS4238', 'CS4239'],  # Security
    ['CS2102', 'CS3223', 'CS4221', 'CS4224', 'CS4225'],            # Database
    ['CS2108', 'CS3245', 'CS4242', 'CS4248', 'CS4347'],            # MIR
    ['CS2105', 'CS3103', 'CS4222', 'CS4226', 'CS4231'],            # Networks
    ['CS3210', 'CS3211', 'CS4231', 'CS4223'],                      # Parallel
    ['CS2104', 'CS3211', 'CS4212', 'CS4215'],                      # Languages
    ['CS2103T', 'CS3213', 'CS3219', 'CS4211', 'CS4218', 'CS4239']  # SWE
]

In [ ]:
# THIS HAS BEEN DONE FOR YOU
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    user = update.effective_user
    await update.message.reply_text(text=f'Hello {user.mention_markdown_v2()}\!'
                                         f'\nEnter /help to see the list of '
                                         f'commands for this bot\.',
                                    parse_mode=ParseMode.MARKDOWN_V2)

async def error(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    print(f'Update {update}  caused error {context.error}')

In [ ]:
# TODO: What should the /help command do?
async def help(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Shows the list of commands this bot has."""
    """We'll deal with this when we're done with everything."""
    await update.message.reply_text(text=f'This shows the list of commands'
                                         f'this bot has.')

In [ ]:
import math

async def quadratic(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    print("Quadratic function called")  # Debugging statement

    # extract the message from the user input
    # e.g. /quadratic 1 2 3
    message = update.message.text
    #print("Received message:", message)  # Debugging statement

    arguments = message.split()  # Split the message by whitespace
    #print("Arguments:", arguments)  # Debugging statement

    terms = arguments[1:]  # Extract the coefficients
    terms = list(map(lambda x: float(x), terms))  # Convert coefficients to floats
    #print("Coefficients:", terms)  # Debugging statement

    a = terms[0]
    b = terms[1]
    c = terms[2]

    # calculate the discriminant
    disc = b**2 - 4*a*c
    #print("Discriminant:", disc)  # Debugging statement

    # Find Roots
    if disc > 0:
        # Two real roots
        root1 = (-b + math.sqrt(disc)) / (2*a)
        root2 = (-b - math.sqrt(disc)) / (2*a)
        reply = f"Your roots are {root1} and {root2}"
    elif disc == 0:
        # One real root (repeated)
        root = -b / (2*a)
        reply = f"Your root is {root}"
    else:
        # Imaginary roots
        real_part = -b / (2*a)
        imag_part = math.sqrt(abs(disc)) / (2*a)
        reply = f"Your roots are {real_part} + {imag_part}i and {real_part} - {imag_part}i"

    #print("Reply:", reply)  # Debugging statement

    await update.message.reply_text(reply)


In [ ]:
async def echo(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    #print("Echo function called")  # Debugging statement
    #print("Received message:", update.message.text)  # Debugging statement
    await update.message.reply_text(update.message.text)


In [ ]:
# TODO: Let's handle any user input without a command associated!
def handle_response(text: str) -> str:
    for focus_area in PRIMARIES_LIST:
      if text in focus_area:
        return display_module(requests.get(f'https://api.nusmods.com/v2/2023-2024/modules/{text}.json').json())
    return 'Sorry, I do not have information on that module...'


async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    message_type: str = update.message.chat.type # Private? Group?
    text: str = update.message.text              # Message content

    # Logging user input on local terminal
    print(f'User ({update.message.chat.id}) in {message_type}: "{text}"')

    # Handle that response!
    response: str = handle_response(text)

    # Logging bot's response on local terminal
    print('Bot:', response)

    # TODO: Make the bot reply with your response!
    await update.message.reply_text(response)

In [ ]:
if __name__ == '__main__':
    # Initializes your bot with the token!
    app = Application.builder().token(TOKEN).build()

    # Commands TODO: Add Handlers for /quadratic and /focusarea (later)
    app.add_handler(CommandHandler('start', start))
    app.add_handler(CommandHandler('help', help))
    app.add_handler(CommandHandler('quadratic', quadratic))
    app.add_handler(CommandHandler('focusarea', focusarea))
    app.add_handler(CommandHandler("echo", echo))


    # Message Handlers
    app.add_handler(MessageHandler(filters.TEXT, handle_message))

    # Callback - we'll come back to this!
    app.add_handler(CallbackQueryHandler(display_focus_area))

    # Error Handlers
    app.add_error_handler(error)

    # Polls the bot
    print('Polling...')
    asyncio.run(app.run_polling())

# **What the API?**
# APIs, or application programming interfaces, are a way for two or more computer programs or components to communicate with each other.
# They can be described as "contracts" that define how separate software interacts with each other.
# REST APIs enable you to develop all kinds of web applications having all possible CRUD (create, retrieve, update, delete) operations.
# REST guidelines suggest using a specific HTTP method on a particular type of call made to the server.
**HTTP Requests**

GET - get data

POST - creating or inserting new data

PUT - update existing data

DELETE - deletes data

In [ ]:
#get the chat ID
def get_chat_id(bot_token):
    url = f"https://api.telegram.org/bot{bot_token}/getUpdates"
    response = requests.get(url)
    data = response.json()
    chat_id = data['result'][0]['message']['chat']['id']
    return chat_id

if __name__ == '__main__':
    bot_token = "7139143567:AAGIdrobGpFaeX4XbdGQfEuRzk7l_VK_LQs"
    chat_id = get_chat_id(bot_token)
    print(f"Your chat ID is: {chat_id}")


In [ ]:
#use the chat to send message
def send_message(bot_token, chat_id, message):
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    data = {'chat_id': chat_id, 'text': message}
    response = requests.post(url, data=data)
    return response.json()

if __name__ == '__main__':
    bot_token = "7139143567:AAGIdrobGpFaeX4XbdGQfEuRzk7l_VK_LQs"  # Replace with your actual bot token
    chat_id = "6145741163"  # Replace with the chat ID you obtained
    message = "HAPPY 520 baby I love u but u need to study moreeee!"
    response = send_message(bot_token, chat_id, message)
    print(response)


In [ ]:
# Using catfact.ninja API
# add /fact to the end of the API url
endpoint = "https://catfact.ninja/fact"

# Sends a GET request to the endpoint
# This calls a random fact
response = requests.get(endpoint)
response # Returns <Response [200]>, meaning successful query!

print(response.text) # prints some random fact about cats in str format

# JSON to dictionary
response = response.json()

print(response) # response is now a dictionary with one key-value pair
print(response['fact']) # prints value of response dictionary with key 'fact'

In [ ]:
def get_news():
    # Replace 'YOUR_API_KEY' with your actual News API key
    api_key = '485566bb090147098c1a04a4f437fae2'
    url = 'https://newsapi.org/v2/top-headlines'  # Example endpoint
    params = {
        'apiKey': api_key,
        'country': 'uk',  # Example: Retrieve headlines from the United States
    }

    response = requests.get(url, params=params)
    data = response.json()

    # Process each article in the response
    for article in data['articles']:
        source = article['source']['name']
        author = article['author']
        title = article['title']
        description = article['description']
        url = article['url']
        published_at = article['publishedAt']
        
        # Print the details of each article
        print(f"Source: {source}")
        print(f"Author: {author}")
        print(f"Title: {title}")
        print(f"Description: {description}")
        print(f"URL: {url}")
        print(f"Published At: {published_at}")
        print("\n")  # New line for better readability between articles

# Example usage
get_news()

In [ ]:
# Using data.gov.sg's carpark API
endpoint = "https://api.data.gov.sg/v1/transport/carpark-availability"

response = requests.get(endpoint)

carpark_data = response.json()["items"][0]["carpark_data"]
len(carpark_data)

# print carpark information for carpark number HE12
for carpark_info in carpark_data:
  if carpark_info["carpark_number"] == "HE12":
    print(carpark_info)
    break

# If you're a CS student, we know you've been starting to wonder about Focus Areas and what you may want to take.
# **Let's make a bot that shows us what Focus Areas there are in CS, and what modules are associated with them.**
# **We'll then be able to search a module and get back some information about the module from NUSMods.**


In [ ]:
# TODO: Go to https://api.nusmods.com/v2/#/ to see how the NUSMods API handles reqs!
# Q1: If I want the title of a module from its code, how would I use the API?
def get_mod_title(module_code: str) -> str:
  endpoint = f'https://api.nusmods.com/v2/2023-2024/modules/{module_code}.json'
  response = requests.get(endpoint)
  description = response.json()
  title = description['title']
  return title

get_mod_title('EL1101E')

In [ ]:
# Q2: When we choose a primary module list from the PRIMARIES_LIST above,
# We want to display the primaries in the chosen focus area in the format:
# (MODULE_CODE) (MODULE_TITLE)
# How would we do that?
def get_primaries_text(primaries_list) -> str:
    result = ""
    for module_code in primaries_list:
      result += f'{module_code} {get_mod_title(module_code)}\n'
    return result

get_primaries_text(['CS3210', 'CS3211', 'CS4231', 'CS4223'])

# When we enter /focusarea to the bot chat, we want to be able to see all the focus areas via INLINE KEYBOARD BUTTONS.


In [ ]:
async def focusarea(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    keyboard = [
        [   # 3 In A Row!
            InlineKeyboardButton(text="Algo & Theory", callback_data="Algorithms and Theory"),
            InlineKeyboardButton(text="AI", callback_data="Artificial Intelligence"),
            InlineKeyboardButton(text="G & G", callback_data="Computer Graphics and Games"),
        ],
        [
            InlineKeyboardButton(text="Security", callback_data="Computer Security"),
            InlineKeyboardButton(text="DB", callback_data="Database Systems"),
            InlineKeyboardButton(text="MIR", callback_data="Multimedia Information Retrieval"),
        ],
        [   # 4 In a Row!
            InlineKeyboardButton(text="Networks", callback_data="Networking and Distributed Systems"),
            InlineKeyboardButton(text="Parallel", callback_data="Parallel Computing"),
            InlineKeyboardButton(text="Prog Langs", callback_data="Programming Languages"),
            InlineKeyboardButton(text="SWE", callback_data="Software Engineering"),
        ]
    ]

    reply_markup = InlineKeyboardMarkup(keyboard)

    await update.message.reply_text(text="Please choose a focus area: ", reply_markup=reply_markup)

# Once the user clicks on a button, we want to change the message to reflect the user's chosen focus area and its primaries in the form of:
# (MODULE_CODE MODULE_TITLE)
# Remember we made a function that provides the string of primaries' codes and titles already with get_primaries_text()!

In [ ]:
def get_primaries(focus_area_title: str) -> str:
    primaries_list = []
    match focus_area_title:
        case "Algorithms and Theory":
            primaries_list = PRIMARIES_LIST[0]
        case "Artificial Intelligence":
            primaries_list = PRIMARIES_LIST[1]
        case "Computer Graphics and Games":
            primaries_list = PRIMARIES_LIST[2]
        case "Computer Security":
            primaries_list = PRIMARIES_LIST[3]
        case "Database Systems":
            primaries_list = PRIMARIES_LIST[4]
        case "Multimedia Information Retrieval":
            primaries_list = PRIMARIES_LIST[5]
        case "Networking and Distributed Systems":
            primaries_list = PRIMARIES_LIST[6]
        case "Parallel Computing":
            primaries_list = PRIMARIES_LIST[7]
        case "Programming Languages":
            primaries_list = PRIMARIES_LIST[8]
        case "Software Engineering":
            primaries_list = PRIMARIES_LIST[9]
        case _:
            primaries_list = []

    return get_primaries_text(primaries_list)


async def display_focus_area(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    # The Inline Keyboard Provides a Callback Query that waits for the user's answer
    query = update.callback_query
    await query.answer()

    # Obtains the primaries text based on the data provided by the Query
    focus_area_primaries = get_primaries(query.data)

    # Changes the 'Please select' message to:
    await query.edit_message_text(text=f"Selected option: <b>{query.data}</b>\n"
                                       f"Enter '(MODULE_CODE)' to find out more:\n\n"
                                       f"Primaries: \n{focus_area_primaries}",
                                  parse_mode=ParseMode.HTML)

# After the user selects a focus area and sees the long list of primaries in that focus area, they want to learn more about the modules without leaving the bot chat.
# Let's add functionality to our bot's message handling so that if we type in a primary module code, our bot will pull information from NUSMods's API to summarise what the user would need to know before signing up.
# 1. Complete the function display_module below that accepts a NUSMods API result regarding a single module in JSON format and returns a string containing information about the module.
# 2. Go back to the handle_response function and modify it so that if the message contains a primary module code, the reply is the result of display_module()
# Make sure to rerun main(), revamp the /help command and update your commands with BotFather with /setcommands!

In [ ]:
# TODO: Complete display_module()
def display_module(module_json) -> str:
    module_code = module_json["moduleCode"]
    module_title = module_json["title"]
    module_description = module_json["description"]
    module_credits = module_json["moduleCredit"]
    module_prereqs = module_json["prerequisite"]

    module_sems = ""
    if "mpes1" in module_json["attributes"]:
      module_sems += "Offered in Sem1\n"
    if "mpes" in module_json["attributes"]:
      module_sems += "Offered in Sem2\n"
    if "su" in module_json["attributes"]:
      module_sems += "Can SU\n"

    message = f'{module_code} {module_title}\n'
    message += f'{module_credits}MCs awarded\n'
    message += f'{module_sems}\n'
    message += f'Prerequisites: {module_prereqs}\n'
    message += f'\nAbout: {module_description}\n'

    return message

endpoint = f'https://api.nusmods.com/v2/2023-2024/modules/EL1101E.json'
response = requests.get(endpoint)
json_response = response.json()
display_module(json_response)

In [47]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://nuscollege.nus.edu.sg/events/"
response = requests.get(url)

if response.status_code == 200:
    html_content = response.text
    print("HTML content fetched successfully.")
    print(html_content)  # Print HTML content for debugging
else:
    print("Failed to fetch HTML content. Status code:", response.status_code)

# Use BeautifulSoup to parse HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find all articles with class 'tribe_events'
event_articles = soup.find_all('article', class_='post-10000092 tribe_events type-tribe_events status-publish hentry tribe_events_cat-admissions-and-outreach tribe_events_cat-student-life cat_admissions-and-outreach cat_student-life')

# Print debugging information
print("Number of event articles found:", len(event_articles))

# Iterate over each event article
for event_article in event_articles:
    # Extract event details
    event_name = event_article.find(class_='tribe-event-title').text.strip()
    event_description = event_article.find(class_='tribe-event-description').text.strip()
    event_date_time = event_article.find(class_='tribe-event-date-time-link').text.strip()
    event_location = event_article.find(class_='tribe-event-venue').text.strip()

    # Print event details
    print("Event Name:", event_name)
    print("Description:", event_description)
    print("Date/Time:", event_date_time)
    print("Location:", event_location)
    print()


In [72]:
#Import the sqlite3 Module
import sqlite3
#Connect to the SQLite Database
conn = sqlite3.connect('events.db')
#Create a Cursor Object
cursor = conn.cursor()
#Define the SQL Command to Create the Events Table
create_table_query = '''
CREATE TABLE IF NOT EXISTS events (
    id INTEGER PRIMARY KEY,
    name TEXT,
    description TEXT,
    date TEXT,
    time TEXT,
    location TEXT
)
'''
#Execute the SQL Command to Create the Events Table:
cursor.execute(create_table_query)
#Define Sample Event Data:
sample_events = [
    ("Asian Undergraduate Symposium 2024", "The Asian Undergraduate Symposium (AUS) is a two-week programme that brings together undergraduates from Singapore, Asian regions, with a special focus on ASEAN participants, for cultural exchange, networking, and capacity development.", "1 July 2024", "13 July 2024", "NUS College, 18 College Avenue East, Singapore"),
    ("TOGETHER NOW!", "Kicking off in July, our very first NOW! program is a Telegram-based event connecting you with peers across various disciplines, especially those in your major with similar academic aspirations. Unite with your team to conquer five rounds of games and snag enticing prizes!", "1 July 2024", "15 August 2024", "NUS College, 18 College Avenue East, Singapore"),
    # Add more sample events here...
]
#Insert Sample Event Data into the Events Table:
for event in sample_events:
    cursor.execute('INSERT INTO events (name, description, date, time, location) VALUES (?, ?, ?, ?, ?)', event)
#Commit Changes and Close the Connection:
conn.commit()
conn.close()

In [73]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('events.db')

# Create a cursor object
cursor = conn.cursor()

# Execute an SQL query to select all events from the 'events' table
cursor.execute('SELECT * FROM events')

# Fetch all rows (events) from the query result
events = cursor.fetchall()

# Print the retrieved events
for event in events:
    print(event)

# Close the cursor and connection
cursor.close()
conn.close()


(1, 'Asian Undergraduate Symposium 2024', 'The Asian Undergraduate Symposium (AUS) is a two-week programme that brings together undergraduates from Singapore, Asian regions, with a special focus on ASEAN participants, for cultural exchange, networking, and capacity development.', '1 July 2024', '13 July 2024', 'NUS College, 18 College Avenue East, Singapore')
(2, 'TOGETHER NOW!', 'Kicking off in July, our very first NOW! program is a Telegram-based event connecting you with peers across various disciplines, especially those in your major with similar academic aspirations. Unite with your team to conquer five rounds of games and snag enticing prizes!', '1 July 2024', '15 August 2024', 'NUS College, 18 College Avenue East, Singapore')
(3, 'Asian Undergraduate Symposium 2024', 'The Asian Undergraduate Symposium (AUS) is a two-week programme that brings together undergraduates from Singapore, Asian regions, with a special focus on ASEAN participants, for cultural exchange, networking, and 